In [16]:
import logging
import os
from typing import List
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import subprocess
from datetime import datetime
import git
AOSP_SYNC_PATH = "/mnt/4846A54B46A53A98/AOSP/"
AOSP_ROOT_MATCHER = "<AOSP_ROOT>/"

logging.basicConfig(
    filename='HISTORYlistener.log',
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

logger = logging.getLogger()
fhandler = logging.FileHandler(filename='run_notebook.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [3]:
# git_repos = []
# for dirpath, dirnames, filenames in os.walk(AOSP_SYNC_PATH):
#     if ".git" in dirnames:
#         git_repos.append(dirpath)

In [8]:
# with open("git_dirs_NORM.txt", "w") as txt_file:
#     for path in git_repos:
#         masked_path = path.replace(AOSP_SYNC_PATH, "<AOSP_ROOT>/")
#         txt_file.write(masked_path + "\n")

In [2]:
def get_project_name(path: str) -> str:
    return path.replace(AOSP_ROOT_MATCHER, "")

def get_java_files_from_path(path: str) -> List[str]:
    buffer = []
    logger.info(f"Checking {path}...")
    for dirpath, dirnames, filenames in os.walk(path):
        java_files_in_dirpath = [f"{dirpath}/{filename}" for filename in filenames if filename.endswith(".java")]
        if java_files_in_dirpath:
            buffer.extend(java_files_in_dirpath)
    return buffer

def save_to_file(path: str, content: List[str], mode="a"):
    with open(path, mode) as txt_file:
        logger.info(f"Saving to {path}")
        txt_file.writelines([line + "\n" for line in content])





In [8]:
def save_report(real_path):
    java_file_list = get_java_files_from_path(real_path)
    if java_file_list:
        save_to_file("aosp_java_files.txt", java_file_list)
    else:
        logger.warning(f"{real_path} does not have java files.")
GIT_DIRS = []
with open("git_dirs_NORM.txt", "r") as txt_file:
    for path in txt_file.readlines():
        real_path = path.strip().replace(AOSP_ROOT_MATCHER, AOSP_SYNC_PATH)
        GIT_DIRS.append(real_path)
def get_project_from_path(java_file_path: str) -> str:
    for project_path in GIT_DIRS:
        if java_file_path.startswith(project_path):
            return project_path
    raise Exception(f"{java_file_path = } does not have a project!")


In [13]:
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = []
    for dir in GIT_DIRS:
        print(f"Submit {dir}")
        fut = executor.submit(save_report, dir)
        fut.add_done_callback(lambda x: print(f"{dir} is done"))
        futures.append(fut)
    print("Running...")
    while not all([f.done() for f in futures]):
        pass
    print("All done.")


Submit /mnt/4846A54B46A53A98/AOSP/.repo/manifests
Submit /mnt/4846A54B46A53A98/AOSP/.repo/repo
Submit /mnt/4846A54B46A53A98/AOSP/art
Submit /mnt/4846A54B46A53A98/AOSP/bionic
Submit /mnt/4846A54B46A53A98/AOSP/bootable/libbootloader
Submit /mnt/4846A54B46A53A98/AOSP/bootable/recovery
/mnt/4846A54B46A53A98/AOSP/bootable/recovery is done
Submit /mnt/4846A54B46A53A98/AOSP/build/bazel
Submit /mnt/4846A54B46A53A98/AOSP/build/bazel_common_rules
Submit /mnt/4846A54B46A53A98/AOSP/build/blueprint
Submit /mnt/4846A54B46A53A98/AOSP/build/make
Submit /mnt/4846A54B46A53A98/AOSP/build/orchestrator
Submit /mnt/4846A54B46A53A98/AOSP/build/pesto
Submit /mnt/4846A54B46A53A98/AOSP/build/release
Submit /mnt/4846A54B46A53A98/AOSP/build/soong
Submit /mnt/4846A54B46A53A98/AOSP/cts
Submit /mnt/4846A54B46A53A98/AOSP/dalvik
Submit /mnt/4846A54B46A53A98/AOSP/developers/build
Submit /mnt/4846A54B46A53A98/AOSP/developers/demos
Submit /mnt/4846A54B46A53A98/AOSP/developers/samples/android
Submit /mnt/4846A54B46A53A98/

## Categorization

In [14]:
df_java_files = pd.DataFrame()
buffer = []
with open("aosp_java_files.txt", "r") as txt_file:
    for line in txt_file.readlines():
        line_stripped = line.strip()
        buffer.append({
            "aosp_project": get_project_from_path(line_stripped),
            "java_file": line_stripped
        })
df_java_files = pd.DataFrame.from_records(buffer)
df_java_files.to_csv("categorized_aosp_java_files.csv")


In [15]:
df_java_files.to_csv("categorized_aosp_java_files.csv", index=False)

In [23]:
# load
df_java_files = pd.read_csv("categorized_aosp_java_files.csv")

## Run BOHR

In [22]:
CHECKPOINTS_PATH = "checkpoints.txt"

def timestamp():
    return int(datetime.now().timestamp() * 1000 // 1)
def load_checkpoints():
    if os.path.exists(CHECKPOINTS_PATH):
        with open(CHECKPOINTS_PATH, 'r') as checkpoint_file:
            lines = checkpoint_file.readlines()
            lines = [line.strip() for line in lines]
            return lines
    else:
        return []
def save_checkpoint(path):
    with open(CHECKPOINTS_PATH, 'a') as checkpoint_file:
        checkpoint_file.write(path + "\n")
    logger.info(f"saved {path}")

def run_bohr(java_file_path, file_name):
    ms = timestamp()
    bohr_report_name = f"{file_name}_{ms}"
    subprocess.call(['java', '-jar', '../bohr.jar', java_file_path, bohr_report_name])
    save_checkpoint(java_file_path)
    if bohr_report_name.endswith(".csv"):
        return bohr_report_name
    return bohr_report_name + ".csv"
def execute_bohr_report_generation(file_test):
    filename = run_bohr(file_test, "./reports/bohr_report")
    df_test = pd.read_csv(filename)
    os.remove(filename)

In [31]:
file_test = df_java_files.loc[0]["java_file"]


execute_bohr_report_generation(file_test)

Running BOHR on /mnt/4846A54B46A53A98/AOSP/build/bazel/examples/android_app/java/com/app/Jni.java


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


True